# Краулеры.
## Обкачиваем сайт с переводами песен
1) http://www.perevod-pesen.ru/


In [1]:
# ИМПОРТЫ

import sqlite3
# подключаемся к базе данных
conn = sqlite3.connect('Artist_bot.db')
c = conn.cursor()

import urllib.request
import requests
from bs4 import BeautifulSoup
import re

import requests
from pprint import pprint
session = requests.session()

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

import random

from collections import Counter

In [2]:
artists_text = """30 Seconds To Mars
Abba
Adele
Ariana Grande
Bhad Bhabie
Billie Eilish
Britney Spears
Bruno Mars
Calvin Harris
Cardi B
Carla’s Dreams
Christina Aguilera
Crystal Castles
David Guetta
Demi Lovato
Ed Sheeran
Eminem
Elvis Presley
Evanescence
Fall Out Boy
Flo Rida
Green Day
Guns N' Roses
Hollywood Undead
Imagine Dragons
J Balvin
Justin Bieber
Kadebostany
Kanye West
Katy Perry
Kesha
Lady Gaga
Lana Del Rey
Lara Fabian
Limp Bizkit
Linkin Park
Little Big
LP
Madonna
Maroon 5
Metallica
Miley Cyrus
Muse
Nickelback
Nicki Minaj
Nirvana
One Direction
One Republic
Panic! At The Disco
Pentatonix
Pink
Placebo
PSY
Queen
Radiohead
Rammstein
Rihanna
Scorpions
Selena Gomez
Shakira
Shawn Mendes
Sia
Skillet
Slipknot
Snoop Dogg
Sunrise Avenue
Taylor Swift
The Beatles
The Rolling Stones
Three Days Grace
twenty one pilots
Zaz"""
artists_low_text = artists_text.lower()

Описание таблицы, составляющей базу данных:

TABLE "artist_bot"
- "track_title"	TEXT,    
- "artist"	TEXT,    
- "text"	TEXT,    
- "translated_text"	TEXT;

Теперь напишем функцию, которая притворяется браузером и скачивает страницы в html-формате:

proxy - https://hidemy.name/ru/proxy-list/?__cf_chl_jschl_tk__=391721aada4a3535a69f9cdc6404b4298b4e903d-1591450563-0-AeIit57IiAUuOH3Y63-G0DgjmkEQvyJpcb-G3GLujM8z15l9tSWvlJmGe_n27AUBpnFOpbCAcxK2LVvKPFyPu9hnOMgMvd30FBvXfJX-7rxUYjzkGc2rM2t7yuuFxUxEG0J8OF6956XO8IFXsI_AQHV7c3s6CgtmVtJhdA_fz_LWUu4s67XzIv2Lo7pfsqOwhuR6VVoXAWBJVDXwCegPfLaXcMBQ_dKCilMmbEePIz4TF2NikkO2sKFcllB2O_jbYROooYr1Ph-ci-KoznY5Y6nLoKsVv9Riv2KvhX1YEmJY

In [3]:
def download_page(page):

    try:
        headers = {'User-Agent': ua.random}
        response = session.get(page, headers=headers)
        html_text = response.text

    except Exception:
        known_proxy_ip = '138.197.157.32'
        proxy = {'http': known_proxy_ip, 'https': known_proxy_ip}
        response = session.get(page, proxies=proxy)
        html_text = response.text


    return html_text

Функция, принимающая список строк; заменяющая системные обозначения на знаки; составляющая из строчек текст:

In [4]:
def good_text(lines_list):
    
    good_lines = []
    
    for line in lines_list:
        if line.startswith('[') and line.endswith(']'):
            lines_list.remove(line)

        elif line.startswith('\n<h2 class=') and line.endswith('</h2>\n'):
            lines_list.remove(line)

        elif len(line) <= 1:
            lines_list.remove(line)
            
        else:
            result = re.sub('&#171;','«', line)
            mid_res = re.sub('&#187;','»', result)
            mid2_res = re.sub('&#8212;','—', mid_res)
            mid3_res = re.sub('&#038;','&', mid2_res)
            mid4_res = re.sub('&#[0-9]+;',' ', mid3_res)
            final_res = re.sub('&#8217;','’', mid4_res)

            good_lines.append(final_res)
            
    good_text = '\n'.join(good_lines)
    
    return good_text

Функция, заполняющая таблицу:

Загвоздка была в том, что на выбранном для скачивания сайте разные страницы внутри устроены немного по-разному, поэтому приходится применять 2 разных метода выкачивания инфы

In [5]:
def fullfill_table_texts(track_link, artist): 
    
    page_text = download_page(track_link)

    try:        
        soup = BeautifulSoup(page_text,'html.parser')
        
        trantxt = soup.find('div', {'class' : 'perevod-right'}) # Переведённый текст
        text_trantxt = trantxt.get_text()
        translated_text = good_text(text_trantxt.split('\n'))

        
        orig = soup.find('div', {'class' : 'tekst-left'}) # Текст
        text_text= orig.get_text()
        text = good_text(text_text.split('\n'))

        
        tit = soup.find('h2', {'class' : 'h2tekst'}) # Название
        high_title = tit.get_text()
        title = high_title.lower()

        c.execute("INSERT INTO all_tracks VALUES (?,?,?,?)", 
                  (title, artist, translated_text, text) )
        conn.commit()
        
    except AttributeError or OSError:
        
        search_cat = re.compile('</div>\n<div class="prvd">(.*?)</div>', flags= re.DOTALL) # Переведённый текст
        tran_lines_list = search_cat.findall(page_text)
        translated_text = good_text(tran_lines_list[1:])
        
        search_cat = re.compile('<div class="stroka">\n<div class="text">(.*?)</div>', flags= re.DOTALL) # Текст
        text_lines_list = search_cat.findall(page_text)
        text = good_text(text_lines_list[1:])
        
        tit = soup.find('h2', {'class' : 'h2tekst'}) # Название
        high_title = tit.get_text()
        title = high_title.lower()

        c.execute("INSERT INTO all_tracks VALUES (?,?,?,?)", 
                  (title, artist, translated_text, text) )
        conn.commit()

Артисты:

In [6]:
artists = artists_low_text.split("\n")
print (artists)
print(len(artists))

['30 seconds to mars', 'abba', 'adele', 'ariana grande', 'bhad bhabie', 'billie eilish', 'britney spears', 'bruno mars', 'calvin harris', 'cardi b', 'carla’s dreams', 'christina aguilera', 'crystal castles', 'david guetta', 'demi lovato', 'ed sheeran', 'eminem', 'elvis presley', 'evanescence', 'fall out boy', 'flo rida', 'green day', "guns n' roses", 'hollywood undead', 'imagine dragons', 'j balvin', 'justin bieber', 'kadebostany', 'kanye west', 'katy perry', 'kesha', 'lady gaga', 'lana del rey', 'lara fabian', 'limp bizkit', 'linkin park', 'little big', 'lp', 'madonna', 'maroon 5', 'metallica', 'miley cyrus', 'muse', 'nickelback', 'nicki minaj', 'nirvana', 'one direction', 'one republic', 'panic! at the disco', 'pentatonix', 'pink', 'placebo', 'psy', 'queen', 'radiohead', 'rammstein', 'rihanna', 'scorpions', 'selena gomez', 'shakira', 'shawn mendes', 'sia', 'skillet', 'slipknot', 'snoop dogg', 'sunrise avenue', 'taylor swift', 'the beatles', 'the rolling stones', 'three days grace', '

Страница каждого исполнителя из списка, с неё выбираются ссылки на страницы всех его треков:

In [7]:
for one in artists:
    first_letter = one[0]
    name = '-'.join(one.split())
    page = 'http://www.perevod-pesen.ru/category/' + first_letter + '/' + (name) + '/'
    try:    
        page_text = download_page(page)
        search_cat = re.compile('''<h2 class="wpex-loop-entry-title">\n<a href="(.*?)" title=''', flags= re.DOTALL)
        all_tracks_links = search_cat.findall(page_text)
        
        for track in all_tracks_links:
            fullfill_table_texts(track, one)

    except Exception:
        print('Ошибка')
        continue

Ошибка
Ошибка
Ошибка
Ошибка
Ошибка


Посмотрела на всё скачанное в БД. Стало понятно, что есть всякие недоработки, подчистила их здесь:

In [8]:
c.execute('SELECT text FROM all_tracks')
for each_one in c.fetchall():
    each_text = each_one[0]
    lines = each_text.split('\n')
    #print(lines)
    if len(lines) == 1 or lines[1] == 'ПОКАЗАТЬ ТЕКСТ ПЕСНИ':
        c.execute('DELETE FROM all_tracks WHERE text=?',(each_text, ))
        conn.commit()

In [9]:
c.execute('SELECT translated_text FROM all_tracks')
for each_one in c.fetchall():
    each_text = each_one[0]
    lines = each_text.split('\n')
    
    if lines[0].startswith('<a href="#a1">'):
        new_lines = lines[1:]
        new_text = '\n'.join(new_lines)    
        c.execute('UPDATE all_tracks SET translated_text = ? WHERE translated_text = ?',(new_text, each_text ))
        conn.commit()

Ура! 697 итоговых треков!